In [1]:
import torch
import os
os.environ['MKL_THREADING_LAYER']='TBB'
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack


/Users/nanwu/anaconda3/envs/deoxys/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
import os.path as osp
import os
import sys
__file__='AxC_GCN_example.ipynb'
root_folder = osp.abspath(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
sys.path.append(root_folder )

In [3]:
from model.Appx_GCN import Appx_GCN
device = "cpu"
data = Dataset(root = '../data', name = 'cora')
adj, features, labels = torch.Tensor(data.adj.toarray()), torch.Tensor(data.features.toarray()), data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

degrees=adj.sum(dim=1)
# Setup Surrogate model
surrogate = GCN(nfeat = features.shape[1], nclass = labels.max().item() + 1,
            nhid = 16, dropout = 0, with_relu = True, with_bias = True, device = device)
surrogate.fit(features, adj, torch.LongTensor(labels), idx_train, idx_val, patience=30)

Loading cora dataset...
Selecting 1 largest connected components


In [4]:
# Setup Attack Model
target_node = 1168
model = Nettack(surrogate, nnodes = adj.shape[0], attack_structure = False, attack_features = True, device = device)
# Attack
model.attack(features, adj, labels, target_node, n_perturbations=int(degrees[1168].item())*2)
modified_adj = torch.Tensor(model.modified_adj.toarray())
modified_features = torch.Tensor(model.modified_features.toarray())

##### Starting attack #####
##### Attack only using feature perturbations #####
##### Attacking the node directly #####
##### Performing 10 perturbations #####
##### ...1/10 perturbations ... #####
##### ...2/10 perturbations ... #####
##### ...3/10 perturbations ... #####
##### ...4/10 perturbations ... #####
##### ...5/10 perturbations ... #####
##### ...6/10 perturbations ... #####
##### ...7/10 perturbations ... #####
##### ...8/10 perturbations ... #####
##### ...9/10 perturbations ... #####
##### ...10/10 perturbations ... #####


In [5]:
# Test
print(torch.exp(surrogate.predict(features, adj))[target_node])
print(torch.exp(surrogate.predict(modified_features, modified_adj))[target_node])

tensor([0.0063, 0.7961, 0.1636, 0.0048, 0.0141, 0.0107, 0.0044],
       grad_fn=<SelectBackward0>)
tensor([0.0046, 0.1767, 0.7952, 0.0042, 0.0114, 0.0062, 0.0018],
       grad_fn=<SelectBackward0>)


In [ ]:
# use approximate model
axc_gcn= Appx_GCN(nfeat = features.shape[1],
              nhid = 16,
              nclass = labels.max().item() + 1,
              dropout = 0.5, device = device, para_path = "../Pre-trained/GCN_Cora.pkl")
axc_gcn.fit(modified_features, modified_adj, labels, idx_train, idx_val, patience = 30, train_iters = 0)

print(torch.exp(axc_gcn.predict(features, adj))[target_node])
print(torch.exp(axc_gcn.predict(modified_features, modified_adj))[target_node])